# Problem set - DEM 


### Stylised facts of the business cycle

In [170]:
pip install fredapi

Note: you may need to restart the kernel to use updated packages.


In [171]:
import pandas as pd
import matplotlib as plt
import numpy as np
import statsmodels.api as sm
from scipy import signal

In [172]:
from fredapi import Fred
fred = Fred(api_key='25400ff00f45200a6b52344af538c912')

# Table 1

## Recreating the table
Getting the values for CND non-durable consumption, CD durable
consumption, H total hours worked, AveH average hours worked per employee, L employment,
GNP/L productivity, AveW average hourly wage based on national accounts.

In [173]:
gnp = fred.get_series('GNP', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q")
gnp.name = 'gnp'
gnp.tail()

2021-01-01    22511.170
2021-04-01    23192.674
2021-07-01    23718.255
2021-10-01    24530.587
2022-01-01    24929.184
Name: gnp, dtype: float64

In [174]:
gnp

1964-01-01      674.880
1964-04-01      683.549
1964-07-01      697.079
1964-10-01      702.017
1965-01-01      723.225
                ...    
2021-01-01    22511.170
2021-04-01    23192.674
2021-07-01    23718.255
2021-10-01    24530.587
2022-01-01    24929.184
Name: gnp, Length: 233, dtype: float64

In [175]:
pcend = fred.get_series('PCEND', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") # non durable consumption
pcend.name = 'pcend'

In [176]:
pcedg = fred.get_series('PCEDG', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") # durable consumption
pcedg.name = 'pcedg'

In [177]:
h = fred.get_series('LNU02032200', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") # Total At Work in Nonagricultural Industries
h.name = 'hours_worked'

In [178]:
ave_h = fred.get_series('AWHMAN', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") #  Average Hours of Work Per Week, Nonagricultural Employment, Household Survey for United States
ave_h.name = 'average_worked'

In [179]:
l = fred.get_series('CE16OV', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") # Employment level
l.name = 'employment'

In [180]:
gnp_l = (gnp / l) + 100 #we added 100 in order to be able to apply log
gnp_l.name = 'productivity'

In [181]:
ave_w = fred.get_series('AHETPI', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") # Average Hourly Earnings of Production and Nonsupervisory Employees
ave_w.name = 'average_wage'

In [182]:
pop = pd.concat([gnp, pcend, h, l, pcedg, ave_h, gnp_l, ave_w], axis = 1)
pop

,gnp,pcend,hours_worked,employment,pcedg,average_worked,productivity,average_wage
1964-01-01,674.880,148.6,6747.0,68614.0,58.0,40.5,100.009836,2.50
1964-04-01,683.549,151.5,6788.0,69402.0,59.5,40.8,100.009849,2.52
1964-07-01,697.079,154.9,5225.0,69480.0,61.3,40.9,100.010033,2.55
1964-10-01,702.017,155.8,7033.0,69710.0,59.4,41.0,100.010071,2.56
1965-01-01,723.225,157.8,6903.0,70188.0,64.9,41.3,100.010304,2.59
...,...,...,...,...,...,...,...,...
2021-01-01,22511.170,3274.4,18605.0,150276.0,1990.9,41.5,100.149799,25.26
2021-04-01,23192.674,3416.4,19199.0,151446.0,2113.5,41.5,100.153142,25.66
2021-07-01,23718.255,3482.1,18968.0,153287.0,2035.0,41.4,100.154731,26.12
2021-10-01,24530.587,3572.1,20676.0,155337.0,2101.6,41.4,100.157919,26.59


In [183]:
# lets change all the values into logs
pop_log = np.log(pop)
pop_log

,gnp,pcend,hours_worked,employment,pcedg,average_worked,productivity,average_wage
1964-01-01,6.514535,5.001258,8.816853,11.136252,4.060443,3.701302,4.605269,0.916291
1964-04-01,6.527298,5.020586,8.822912,11.147671,4.085976,3.708682,4.605269,0.924259
1964-07-01,6.546899,5.042780,8.561210,11.148794,4.115780,3.711130,4.605271,0.936093
1964-10-01,6.553958,5.048573,8.858369,11.152099,4.084294,3.713572,4.605271,0.940007
1965-01-01,6.583720,5.061328,8.839711,11.158933,4.172848,3.720862,4.605273,0.951658
...,...,...,...,...,...,...,...,...
2021-01-01,10.021767,8.093890,9.831186,11.920229,7.596342,3.725693,4.606667,3.229222
2021-04-01,10.051592,8.136343,9.862613,11.927984,7.656101,3.725693,4.606700,3.244933
2021-07-01,10.074000,8.155391,9.850509,11.940067,7.618251,3.723281,4.606716,3.262701
2021-10-01,10.107676,8.180909,9.936729,11.953352,7.650454,3.723281,4.606748,3.280535


In [184]:
tabella = pop_log.transpose()
tabella

,1964-01-01,1964-04-01,1964-07-01,1964-10-01,1965-01-01,1965-04-01,1965-07-01,1965-10-01,1966-01-01,1966-04-01,...,2019-10-01,2020-01-01,2020-04-01,2020-07-01,2020-10-01,2021-01-01,2021-04-01,2021-07-01,2021-10-01,2022-01-01
gnp,6.514535,6.527298,6.546899,6.553958,6.583720,6.601002,6.626156,6.654982,6.685526,6.697075,...,9.997050,9.989391,9.893738,9.978708,9.992751,10.021767,10.051592,10.074000,10.107676,10.123794
pcend,5.001258,5.020586,5.042780,5.048573,5.061328,5.080161,5.100476,5.138735,5.159630,5.177843,...,8.004933,8.018593,7.957772,8.039286,8.038609,8.093890,8.136343,8.155391,8.180909,8.205027
hours_worked,8.816853,8.822912,8.561210,8.858369,8.839711,8.834337,8.623174,8.939974,8.941545,8.909235,...,9.990766,9.974691,9.568854,9.747886,9.848503,9.831186,9.862613,9.850509,9.936729,9.930276
employment,11.136252,11.147671,11.148794,11.152099,11.158933,11.168983,11.175619,11.182016,11.186821,11.192638,...,11.974456,11.967695,11.831772,11.892745,11.917817,11.920229,11.927984,11.940067,11.953352,11.968323
pcedg,4.060443,4.085976,4.115780,4.084294,4.172848,4.171306,4.200205,4.235555,4.280824,4.244200,...,7.343362,7.310149,7.304180,7.493596,7.500419,7.596342,7.656101,7.618251,7.650454,7.688822
average_worked,3.701302,3.708682,3.711130,3.713572,3.720862,3.718438,3.716008,3.720862,3.728100,3.728100,...,3.723281,3.720862,3.668677,3.713572,3.720862,3.725693,3.725693,3.723281,3.723281,3.723281
productivity,4.605269,4.605269,4.605271,4.605271,4.605273,4.605274,4.605276,4.605278,4.605281,4.605282,...,4.606554,4.606552,4.606609,4.606644,4.606628,4.606667,4.606700,4.606716,4.606748,4.606750
average_wage,0.916291,0.924259,0.936093,0.940007,0.951658,0.963174,0.970779,0.982078,0.989541,1.000632,...,3.170106,3.180135,3.217675,3.209229,3.218076,3.229222,3.244933,3.262701,3.280535,3.295466


In [185]:
df_list = [gnp, pcend, pcedg, h, ave_h, l, gnp_l, ave_w]
std_values = []
for col in pop_log:
    std_value = pop_log.std()
    std_values.append(std_value)

In [186]:
std_value

gnp               1.057231
pcend             0.912862
hours_worked      0.323363
employment        0.243124
pcedg             1.009104
average_worked    0.017740
productivity      0.000422
average_wage      0.676292
dtype: float64

In [187]:
corr_list = []
lag_values = range(-4, 5)
gdp_t = df_list[0].iloc[:]

for i in df_list:
    for lag in lag_values:
        lagged_variable = i.iloc[:].shift(lag)
        
        if lag < 0:
            aligned_gdp_t = gdp_t.iloc[-lag:]
            aligned_variable = lagged_variable[:len(gdp_t)-lag]
        elif lag > 0:
            aligned_gdp_t = gdp_t.iloc[:-lag]
            aligned_variable = lagged_variable[lag:]
        else:
            aligned_gdp_t = gdp_t
            aligned_variable = lagged_variable
        
        corr = aligned_variable.corr(aligned_gdp_t)
        corr_list.append(corr)

In [188]:
labels = ['gnp','pcend','hours_worked','employment','pcedg','average_worked','productivity','average_wage']

In [189]:
corr_gnp= [labels[0]]+[std_value[0]]+corr_list[0:9]
corr_pcend=[labels[1]]+[std_value[1]]+corr_list[9:18]
corr_h_w=[labels[2]]+[std_value[2]]+corr_list[18:27]
corr_emp=[labels[3]]+[std_value[3]]+corr_list[27:36]
corr_pcedg=[labels[4]]+[std_value[4]]+corr_list[36:45]
corr_avg_w=[labels[5]]+[std_value[5]]+corr_list[45:54]
corr_prod=[labels[6]]+[std_value[6]]+corr_list[54:63]
corr_ave_w=[labels[7]]+[std_value[7]]+corr_list[63:72]

In [190]:
df_tab = [corr_gnp] + [corr_pcend] + [corr_h_w] +[corr_emp] + [corr_pcedg] + [corr_avg_w] + [corr_prod] + [corr_ave_w]
tab1 = pd.DataFrame(df_tab,columns=['Variables','Sd%','t-4','t-3','t-2','t-1','t','t+1','t+2','t+3','t+4'])
tab1

,Variables,Sd%,t-4,t-3,t-2,t-1,t,t+1,t+2,t+3,t+4
0,gnp,1.057231,0.998495,0.998867,0.999245,0.999562,1.000000,0.999554,0.999280,0.999034,0.999170
1,pcend,0.912862,0.996428,0.996903,0.997464,0.997854,0.998402,0.998105,0.997867,0.997596,0.997639
2,hours_worked,0.323363,0.980011,0.981120,0.982575,0.983993,0.986116,0.987465,0.988623,0.988963,0.988918
3,employment,0.243124,0.872154,0.872304,0.873157,0.873122,0.874195,0.873008,0.877590,0.882876,0.899373
4,pcedg,1.009104,0.599679,0.591060,0.582967,0.580945,0.584805,0.581852,0.579134,0.575882,0.613550
5,average_worked,0.017740,0.933937,0.934243,0.934298,0.934162,0.933949,0.934431,0.936691,0.939321,0.945304
6,productivity,0.000422,0.996635,0.996826,0.997024,0.997023,0.997158,0.997278,0.997248,0.997253,0.997170
7,average_wage,0.676292,0.992439,0.992500,0.992519,0.992462,0.992299,0.992226,0.992154,0.992130,0.992044


# Table 2

## Recreating the table
Getting the values for Y per capita output, C per capita consumption, I per capita investment, N
per capita hours, w the real wage (compensation per hour), r the real interest rate, and A total
factor productivity.

In [191]:
y = fred.get_series('PRS85006161', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") # Output per capita
y.name = 'output_per_capita'

In [192]:
c = fred.get_series('A794RX0Q048SBEA', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") # Consumption per capita
c.name = 'consumption_per_capita'

In [193]:
i = fred.get_series('A795RC0Q052SBEA', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") # Investment per capita
i.name = 'investment_per_capita'

In [194]:
n = fred.get_series('AWHMAN', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") #  Average Hours of Work Per Week, Nonagricultural Employment, Household Survey for United States
n.name = 'hours_per_capita'

In [195]:
r = fred.get_series('DGS10', observation_start = '1964-01-01', observation_end = '2022-01-01', frequency = "q") # Real interest filter
r.name = 'real_interest_rate'

In [196]:
a = gnp_l
a.name = 'total_factor_productivity'

In [197]:
r = r.astype(float)

In [198]:
#we create a dataframe that contains the interest rate
pap_r = pd.concat([y, c, i, n, a, r], axis = 1)
pap_r

,output_per_capita,consumption_per_capita,investment_per_capita,hours_per_capita,total_factor_productivity,real_interest_rate
1964-01-01,5.3,12179.0,304.0,40.5,100.009836,4.18
1964-04-01,5.1,12355.0,310.0,40.8,100.009849,4.20
1964-07-01,3.7,12537.0,319.0,40.9,100.010033,4.19
1964-10-01,2.6,12529.0,308.0,41.0,100.010071,4.17
1965-01-01,2.8,12773.0,336.0,41.3,100.010304,4.20
...,...,...,...,...,...,...
2021-01-01,6.6,40323.0,5997.0,41.5,100.149799,1.34
2021-04-01,4.8,41468.0,6363.0,41.5,100.153142,1.59
2021-07-01,0.5,41730.0,6121.0,41.4,100.154731,1.32
2021-10-01,1.7,42014.0,6315.0,41.4,100.157919,1.53


In [199]:
#we decided to add 100 to all values in order to be able to apply the log
pap_r = pap_r + 100
pap_r

,output_per_capita,consumption_per_capita,investment_per_capita,hours_per_capita,total_factor_productivity,real_interest_rate
1964-01-01,105.3,12279.0,404.0,140.5,200.009836,104.18
1964-04-01,105.1,12455.0,410.0,140.8,200.009849,104.20
1964-07-01,103.7,12637.0,419.0,140.9,200.010033,104.19
1964-10-01,102.6,12629.0,408.0,141.0,200.010071,104.17
1965-01-01,102.8,12873.0,436.0,141.3,200.010304,104.20
...,...,...,...,...,...,...
2021-01-01,106.6,40423.0,6097.0,141.5,200.149799,101.34
2021-04-01,104.8,41568.0,6463.0,141.5,200.153142,101.59
2021-07-01,100.5,41830.0,6221.0,141.4,200.154731,101.32
2021-10-01,101.7,42114.0,6415.0,141.4,200.157919,101.53


In [200]:
#now we can apply the log
pap_log = np.log(pap_r)
pap_log

,output_per_capita,consumption_per_capita,investment_per_capita,hours_per_capita,total_factor_productivity,real_interest_rate
1964-01-01,4.656813,9.415646,6.001415,4.945207,5.298367,4.646120
1964-04-01,4.654912,9.429877,6.016157,4.947340,5.298367,4.646312
1964-07-01,4.641502,9.444384,6.037871,4.948050,5.298368,4.646216
1964-10-01,4.630838,9.443751,6.011267,4.948760,5.298368,4.646024
1965-01-01,4.632785,9.462887,6.077642,4.950885,5.298369,4.646312
...,...,...,...,...,...,...
2021-01-01,4.669084,10.607154,8.715552,4.952300,5.299066,4.618481
2021-04-01,4.652054,10.635086,8.773849,4.952300,5.299083,4.620945
2021-07-01,4.610158,10.641369,8.735686,4.951593,5.299091,4.618284
2021-10-01,4.622027,10.648136,8.766394,4.951593,5.299107,4.620354


In [201]:
#Here in the output, we can see that we have separated the cycle and trend component from the time series. 
#we will need just the cycle component of each variable so we are going to apply the hp filter to each of it.

from statsmodels.tsa.filters.hp_filter import hpfilter

cycle,trend = hpfilter(pap_log['output_per_capita'], lamb=1600)
analysis = pap_log[['output_per_capita']]
analysis['cycle']= cycle
analysis['trend'] = trend
analysis



/var/folders/9f/rqj70_jx7sdfslg63hq6syh80000gn/T/ipykernel_55721/3587413457.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis['cycle']= cycle
/var/folders/9f/rqj70_jx7sdfslg63hq6syh80000gn/T/ipykernel_55721/3587413457.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis['trend'] = trend


,output_per_capita,cycle,trend
1964-01-01,4.656813,0.009007,4.647807
1964-04-01,4.654912,0.009104,4.645808
1964-07-01,4.641502,-0.002313,4.643815
1964-10-01,4.630838,-0.011001,4.641838
1965-01-01,4.632785,-0.007104,4.639889
...,...,...,...
2021-01-01,4.669084,0.039368,4.629716
2021-04-01,4.652054,0.021865,4.630189
2021-07-01,4.610158,-0.020408,4.630566
2021-10-01,4.622027,-0.008863,4.630890


In [202]:
# Define the lambda value for the HP filter (a value of 1600 is suggested for quarterly data.)
lambda_value = 1600 
# we create an empty DataFrame to store the detrended variables
pap_detrended = pd.DataFrame()

# we must iterate over each column in our pap DataFrame and apply the HP filter to the current column. 
#then we are going to add the detrended column to the new pap_detrend
for column in pap_log.columns:
    detrended, _ = sm.tsa.filters.hpfilter(pap_log[column], lamb=lambda_value)
    pap_detrended[column] = detrended


print(pap_detrended)

            output_per_capita  consumption_per_capita  investment_per_capita  \
1964-01-01           0.009007               -0.012304              -0.006072   
1964-04-01           0.009104               -0.007585              -0.004375   
1964-07-01          -0.002313               -0.002582               0.004298   
1964-10-01          -0.011001               -0.012700              -0.035337   
1965-01-01          -0.007104               -0.003014               0.018022   
...                       ...                     ...                    ...   
2021-01-01           0.039368               -0.001994               0.072287   
2021-04-01           0.021865                0.020754               0.105710   
2021-07-01          -0.020408                0.021771               0.042473   
2021-10-01          -0.008863                0.023231               0.048003   
2022-01-01          -0.038093                0.020663               0.060117   

            hours_per_capita  total_fac

### Standard deviation

In [203]:
std_dev_table2 = pap_detrended.std()
std_dev_table2 

output_per_capita            0.016309
consumption_per_capita       0.014141
investment_per_capita        0.037221
hours_per_capita             0.002731
total_factor_productivity    0.000004
real_interest_rate           0.006606
dtype: float64

In [204]:
dict1 = {'output_per_capita': [0.016309],
       'consumption_per_capita': [0.014141],
       'investment_per_capita': [  0.037221],
       'hours_per_capita': [0.002731],
         'total_factor_productivity': [0.000004],
       'real_interest_rate': [ 0.006606]}
std_data = pd.DataFrame(dict1)
std_data.index = ['standard_deviation']
std_d = std_data.transpose()
std_d

,standard_deviation
output_per_capita,0.016309
consumption_per_capita,0.014141
investment_per_capita,0.037221
hours_per_capita,0.002731
total_factor_productivity,0.000004
real_interest_rate,0.006606


### Relative standard deviation

In [205]:
std_output_per_capita = 0.016309

In [206]:
rstd_dev = (std_dev_table2) / std_output_per_capita 
rstd_dev

output_per_capita            1.000008
consumption_per_capita       0.867051
investment_per_capita        2.282217
hours_per_capita             0.167425
total_factor_productivity    0.000234
real_interest_rate           0.405055
dtype: float64

In [207]:
dict2 = {'output_per_capita': [ 1.000000],
       'consumption_per_capita': [ 0.867051],
       'investment_per_capita': [2.282217],
       'hours_per_capita': [ 0.167425],
     'total_factor_productivity'  :[0.000234],
       'real_interest_rate': [0.405055]}
rstd_data = pd.DataFrame(dict2)
rstd_data.index = ['relative_standard_deviation']
rstd_d = rstd_data.transpose()
rstd_d

,relative_standard_deviation
output_per_capita,1.000000
consumption_per_capita,0.867051
investment_per_capita,2.282217
hours_per_capita,0.167425
total_factor_productivity,0.000234
real_interest_rate,0.405055


### First Order AutoCorrelation
Plotting the correlation between the variables from 1964 to 2022

In [208]:
#We created a function that computes the autocorrelation for each column of our dataframe

def df_autocorr(df, lag=1, axis=0):
  
    return df.apply(lambda col: col.autocorr(lag), axis=axis)

autocorr_values = df_autocorr(pap_detrended)

print(autocorr_values)

output_per_capita            0.730247
consumption_per_capita       0.716760
investment_per_capita        0.732871
hours_per_capita             0.714220
total_factor_productivity    0.669984
real_interest_rate           0.795981
dtype: float64


In [209]:
dict3 = {'output_per_capita': [ 0.730247],
       'consumption_per_capita': [0.716760],
       'investment_per_capita': [0.732871],
       'hours_per_capita': [0.714220],
     'total_factor_productivity'  :[0.669984],
       'real_interest_rate': [0.795981]}
rautocorr_data = pd.DataFrame(dict3)
rautocorr_data.index = ['first_order_autocorr']

rautocorr = rautocorr_data.transpose()
rautocorr

,first_order_autocorr
output_per_capita,0.730247
consumption_per_capita,0.716760
investment_per_capita,0.732871
hours_per_capita,0.714220
total_factor_productivity,0.669984
real_interest_rate,0.795981


### Contemporanous correlation

In [210]:
columns_to_correlate = ['output_per_capita', 
                        'consumption_per_capita', 
                        'investment_per_capita', 
                        'hours_per_capita',
                        'total_factor_productivity',
                        'real_interest_rate']
for column in columns_to_correlate:
    correlation = pap_detrended['output_per_capita'].corr(pap_detrended[column])
    print(correlation)

1.0
0.07621118877259442
0.20931208045769595
0.2724034367376957
0.1702503817845949
-0.1420303790133451


In [211]:
dict4 = {'output_per_capita': [1.0],
       'consumption_per_capita': [0.07621118877259442],
       'investment_per_capita': [0.20931208045769595],
       'hours_per_capita': [0.2724034367376957],
    'total_factor_productivity': [0.1702503817845949],
       'real_interest_rate': [-0.1420303790133451]}
corr_data = pd.DataFrame(dict4)
corr_data.index = ['correlation_with_output']
corr_data
hello = corr_data.transpose()
hello

,correlation_with_output
output_per_capita,1.000000
consumption_per_capita,0.076211
investment_per_capita,0.209312
hours_per_capita,0.272403
total_factor_productivity,0.170250
real_interest_rate,-0.142030


### Final table 2

In [212]:
final2 = pd.concat([std_d, rstd_d, rautocorr, hello], axis = 1)
final2

,standard_deviation,relative_standard_deviation,first_order_autocorr,correlation_with_output
output_per_capita,0.016309,1.000000,0.730247,1.000000
consumption_per_capita,0.014141,0.867051,0.716760,0.076211
investment_per_capita,0.037221,2.282217,0.732871,0.209312
hours_per_capita,0.002731,0.167425,0.714220,0.272403
total_factor_productivity,0.000004,0.000234,0.669984,0.170250
real_interest_rate,0.006606,0.405055,0.795981,-0.142030
